In [127]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt

In [128]:
# Load the data
df = pd.read_csv('/home/tom/Python/Machine learning/pytorch/rnn/XAU_USD Historical Data3.csv' , thousands=',')

In [129]:
# Extract 'Date' and 'Close' columns
df = df[['Date', 'Open', 'Price']]

# Convert 'Date' to datetime and sort by date
df['Date'] = pd.to_datetime(df['Date'])
df.sort_values('Date', inplace=True)
df

,Date,Open,Price
6211,2000-02-02,282.88,284.88
6210,2000-02-03,285.00,287.40
6209,2000-02-04,287.75,315.20
6208,2000-02-07,310.00,301.80
6207,2000-02-08,302.25,299.00
...,...,...,...
4,2023-12-05,2030.15,2019.42
3,2023-12-06,2019.74,2024.90
2,2023-12-07,2025.30,2028.34
1,2023-12-08,2028.69,2003.39


In [130]:
# Normalize the 'Open' and 'Price' values
scaler_open = MinMaxScaler()
scaler_price = MinMaxScaler()

# The columns 'Open' and ''Price is changed to range between -1 and +1
df['Open'] = scaler_open.fit_transform(df['Open'].values.reshape(-1, 1))
df['Price'] = scaler_price.fit_transform(df['Price'].values.reshape(-1, 1))

# Convert the DataFrame to a PyTorch tensor and add one additional dimension
features = torch.tensor(df[['Open', 'Price']].values.astype(np.float32))
features.shape

torch.Size([6212, 2])

In [131]:
# Define a function to create input sequences and corresponding targets
def create_sequences(data, seq_length):
    sequences = []
    targets_open = []
    targets_price = []
    for i in range(len(data) - seq_length):
        seq = data[i:i+seq_length]
        target_open = data[i+seq_length:i+seq_length+1, 0]  # Assuming 'Open' is the first column
        target_price = data[i+seq_length:i+seq_length+1, 1]  # Assuming 'Price' is the second column
        sequences.append(seq)
        targets_open.append(target_open)
        targets_price.append(target_price)
    return torch.stack(sequences), torch.stack(targets_open), torch.stack(targets_price)

In [132]:
# Define hyperparameters
sequence_length = 10
batch_size = 64
epochs = 100
learning_rate = 0.001

In [133]:
# Create sequences and targets
sequences, targets_open, targets_price = create_sequences(features, sequence_length)

# Split the data into training and testing sets
split_ratio = 0.8
split_index = int(split_ratio * len(sequences))

train_sequences, test_sequences = sequences[:split_index], sequences[split_index:]
train_targets_open, test_targets_open = targets_open[:split_index], targets_open[split_index:]
train_targets_price, test_targets_price = targets_price[:split_index], targets_price[split_index:]

In [134]:
print('X shape ', sequences.shape)
print('open shape ', targets_open.shape)
print('price shape ', targets_price.shape)

X shape  torch.Size([6202, 10, 2])
open shape  torch.Size([6202, 1])
price shape  torch.Size([6202, 1])


In [135]:
# Create DataLoader for training
train_dataset_open = TensorDataset(train_sequences, train_targets_open)
train_dataloader_open = DataLoader(train_dataset_open, batch_size=batch_size, shuffle=True)
train_dataset_price = TensorDataset(train_sequences, train_targets_price)
train_dataloader_price = DataLoader(train_dataset_price, batch_size=batch_size, shuffle=True)

In [136]:
# Test the model on the testing set
test_dataset_open = TensorDataset(test_sequences, test_targets_open)
test_dataloader_open = DataLoader(test_dataset_open, batch_size=batch_size, shuffle=True)
test_dataset_price = TensorDataset(test_sequences, test_targets_price)
test_dataloader_price = DataLoader(test_dataset_price, batch_size=batch_size, shuffle=True)

In [137]:
# Define LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_size=2, hidden_size=100, num_layers=1, output_size=1):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out

In [138]:
# Instantiate the model
model_open = LSTMModel()
model_price = LSTMModel()


# Define loss and optimizer for 'Open'
criterion_open = nn.MSELoss()
optimizer_open = torch.optim.Adam(model_open.parameters(), lr=learning_rate)

# Train the model
for epoch in range(epochs):
    for batch_seq, batch_target in train_dataloader_open:
        optimizer_open.zero_grad()
        output_open = model_open(batch_seq)
        loss_open = criterion_open(output_open, batch_target)
        loss_open.backward()
        optimizer_open.step()
    print(f'Epoch {epoch+1}/{epochs}, Loss Open: {loss_open.item()}')

# Define loss and optimizer for 'Price'
criterion_price = nn.MSELoss()
optimizer_price = torch.optim.Adam(model_price.parameters(), lr=learning_rate)

for epoch in range(epochs):
    for batch_seq, batch_target in train_dataloader_price:
        optimizer_price.zero_grad()
        output_price = model_price(batch_seq)
        loss_price = criterion_price(output_price, batch_target)
        loss_price.backward()
        optimizer_price.step()
    print(f'Epoch {epoch+1}/{epochs}, Loss Price: {loss_price.item()}')



Epoch 1/100, Loss Open: 0.0007475613965652883
Epoch 2/100, Loss Open: 8.371967851417139e-05
Epoch 3/100, Loss Open: 8.94896438694559e-05
Epoch 4/100, Loss Open: 5.683614654117264e-05
Epoch 5/100, Loss Open: 0.00013751137885265052
Epoch 6/100, Loss Open: 5.4234904382610694e-05
Epoch 7/100, Loss Open: 0.00011574140808079392
Epoch 8/100, Loss Open: 2.9957627702970058e-05
Epoch 9/100, Loss Open: 7.851084228605032e-05
Epoch 10/100, Loss Open: 7.178817759267986e-05
Epoch 11/100, Loss Open: 9.955629502655938e-05
Epoch 12/100, Loss Open: 4.961801096214913e-05
Epoch 13/100, Loss Open: 7.331501547014341e-05
Epoch 14/100, Loss Open: 0.0001369682577205822
Epoch 15/100, Loss Open: 7.575711060781032e-05
Epoch 16/100, Loss Open: 0.00010989492147928104
Epoch 17/100, Loss Open: 7.499847561120987e-05
Epoch 18/100, Loss Open: 4.6834749809931964e-05
Epoch 19/100, Loss Open: 7.577694486826658e-05
Epoch 20/100, Loss Open: 6.26996552455239e-05
Epoch 21/100, Loss Open: 4.9458649300504476e-05
Epoch 22/100, Los

In [146]:
# Make predictions for the next 10 days
with torch.no_grad():
    future_values_open = features[-sequence_length:, 0].clone()
    future_values_price = features[-sequence_length:, 1].clone()

    for i in range(10):
        input_sequence_open = future_values_open[-sequence_length:].view(1, -1, 1)
        input_sequence_price = future_values_price[-sequence_length:].view(1, -1, 1)

        prediction_open = model_open(input_sequence_open)
        prediction_price = model_price(input_sequence_price)

        future_values_open = torch.cat([future_values_open, prediction_open.view(-1)])
        future_values_price = torch.cat([future_values_price, prediction_price.view(-1)])



# Inverse transform the predicted values
predicted_values_open = scaler_open.inverse_transform(future_values_open.numpy().reshape(-1, 1))
predicted_values_price = scaler_price.inverse_transform(future_values_price.numpy().reshape(-1, 1))

# Check and adjust lengths
min_length_open = min(len(future_dates), len(predicted_values_open))
min_length_price = min(len(future_dates), len(predicted_values_price))
future_dates = future_dates[:min_length_open]

# Flatten the 2D arrays
predicted_values_open_flat = predicted_values_open.flatten()
predicted_values_price_flat = predicted_values_price.flatten()

# Create DataFrames for the predicted values
predicted_df_open = pd.DataFrame({
    'Date': future_dates,
    'Predicted_Open': predicted_values_open_flat[:min_length_open]
})

predicted_df_price = pd.DataFrame({
    'Date': future_dates,
    'Predicted_Price': predicted_values_price_flat[:min_length_price]
})

# Print the predicted DataFrames
print(predicted_df_open)
print(predicted_df_price)


        Date  Predicted_Open
0 2023-12-12     2013.939941
1 2023-12-13     2041.239990
2 2023-12-14     2044.800049
3 2023-12-15     2034.489868
4 2023-12-18     2071.070068
5 2023-12-19     2030.150024
6 2023-12-20     2019.739868
7 2023-12-21     2025.300171
8 2023-12-22     2028.689941
        Date  Predicted_Price
0 2023-12-12      2040.890015
1 2023-12-13      2044.590088
2 2023-12-14      2035.750122
3 2023-12-15      2070.899902
4 2023-12-18      2030.059937
5 2023-12-19      2019.420166
6 2023-12-20      2024.899902
7 2023-12-21      2028.339966
8 2023-12-22      2003.390015
